In [1]:
import sys
sys.path.append('..') # Search for custom module in the top level. 

# Import my custom modules.
from allens_quantum_package.functions import * 
from allens_quantum_package.operators import *

from qiskit import *
from qiskit.quantum_info import state_fidelity
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_experiments.framework import ExperimentData
from qiskit_experiments.library.tomography import MitigatedTomographyAnalysis, MitigatedStateTomography, TomographyAnalysis
from qiskit_experiments.library.tomography.basis import PauliMeasurementBasis
from qiskit_experiments.library.characterization.analysis import LocalReadoutErrorAnalysis
from qiskit_ibm_runtime import QiskitRuntimeService

from numpy import set_printoptions, radians, ndarray
import scipy

In [2]:
# Set the floating point diplay precision to 2 decimal places, sufficient for our purposes.
set_printoptions(precision=2)

# Initialise the Qiskit runtime service. 
service = QiskitRuntimeService()

In [6]:
states = [
    (radians(-88), radians(79)),
    (radians(32), radians(-167)),
    (radians(140), radians(125)),
    
    (radians(-76), radians(-88)),
    (radians(93), radians(-135)),
    (radians(104), radians(-96)),

    (radians(-104), radians(-146)),
    (radians(-158), radians(-108)),
    (radians(-110), radians(-172))
]

In [3]:
def build_circuit(theta, phi) -> QuantumCircuit:
    circ = QuantumCircuit(1)
    circ.u(theta, phi, 0, 0)
    return circ

In [4]:
ibm_brisbane = service.get_backend('ibm_brisbane')

In [7]:
circuits_to_send = list(itertools.chain.from_iterable([
        circuit for circuit in [
            get_tomography_circuits(build_circuit(theta, phi), 0) for theta, phi in states
        ]
    ]
))

In [ ]:
circuits_to_send = [transpile(circuit, ibm_brisbane) for circuit in circuits_to_send]

In [9]:
# QDT X Circuits
qdt_circ_0_x = QuantumCircuit(1)
qdt_circ_0_x.h(0)
qdt_circ_0_x.h(0)
qdt_circ_0_x.measure_all()

qdt_circ_1_x = QuantumCircuit(1)
qdt_circ_1_x.h(0)
qdt_circ_1_x.z(0)
qdt_circ_1_x.h(0)
qdt_circ_1_x.measure_all()

# QDT Y Circuits
qdt_circ_0_y = QuantumCircuit(1)
qdt_circ_0_y.h(0)
qdt_circ_0_y.s(0)
qdt_circ_0_y.sdg(0)
qdt_circ_0_y.h(0)
qdt_circ_0_y.measure_all()

qdt_circ_1_y = QuantumCircuit(1)
qdt_circ_1_y.h(0)
qdt_circ_1_y.sdg(0)
qdt_circ_1_y.sdg(0)
qdt_circ_1_y.h(0)
qdt_circ_1_y.measure_all()

# QDT Z Circuits
qdt_circ_0_z = QuantumCircuit(1)
qdt_circ_0_z.measure_all()

qdt_circ_1_z = QuantumCircuit(1)
qdt_circ_1_z.x(0)
qdt_circ_1_z.measure_all()

qdt_circuits = [qdt_circ_0_x, qdt_circ_1_x, qdt_circ_0_y, qdt_circ_1_y, qdt_circ_0_z, qdt_circ_1_z]
qdt_circuits = [transpile(circuit, ibm_brisbane, optimization_level=0) for circuit in qdt_circuits]

circuits_to_send = qdt_circuits + circuits_to_send

In [10]:
hardware_job = ibm_brisbane.run(circuits=circuits_to_send)

In [11]:
print(f"Hardware job ID: {hardware_job.job_id()}")

Hardware job ID: cwqn0ehehebg008hxd1g


---

In [13]:
hardware_job = service.job('cwqn0ehehebg008hxd1g')
result_counts = hardware_job.result().get_counts()

In [16]:
result_counts

[{'0': 3770, '1': 230},
 {'1': 3946, '0': 54},
 {'0': 3769, '1': 231},
 {'1': 3947, '0': 53},
 {'0': 3739, '1': 261},
 {'1': 3939, '0': 61},
 {'0': 1555, '1': 2445},
 {'1': 3905, '0': 95},
 {'1': 2038, '0': 1962},
 {'1': 3046, '0': 954},
 {'0': 1698, '1': 2302},
 {'0': 3479, '1': 521},
 {'1': 2772, '0': 1228},
 {'0': 2867, '1': 1133},
 {'1': 3517, '0': 483},
 {'1': 2144, '0': 1856},
 {'0': 3713, '1': 287},
 {'0': 2383, '1': 1617},
 {'1': 3392, '0': 608},
 {'0': 581, '1': 3419},
 {'0': 1830, '1': 2170},
 {'0': 1694, '1': 2306},
 {'1': 3875, '0': 125},
 {'1': 2552, '0': 1448},
 {'0': 3427, '1': 573},
 {'1': 1070, '0': 2930},
 {'1': 2545, '0': 1455},
 {'1': 1824, '0': 2176},
 {'0': 2566, '1': 1434},
 {'1': 3802, '0': 198},
 {'0': 3635, '1': 365},
 {'0': 2149, '1': 1851},
 {'1': 2738, '0': 1262}]

In [17]:
qdt_x_counts = result_counts[0:2]
qdt_y_counts = result_counts[2:4]
qdt_z_counts = result_counts[4:6]

state_1_counts = result_counts[6:9]
state_2_counts = result_counts[9:12]
state_3_counts = result_counts[12:15]
state_4_counts = result_counts[15:18]
state_5_counts = result_counts[18:21]
state_6_counts = result_counts[21:24]
state_7_counts = result_counts[24:27]
state_8_counts = result_counts[27:30]
state_9_counts = result_counts[30:33]

In [19]:
def print_fidelities(x_counts, y_counts, z_counts, theta, phi):

    den_op = density_op_from_counts_dict(x_counts, y_counts, z_counts)
    fidelity = state_fidelity(den_op, gen_qubit(theta, phi))

    print(f'State fidelity for \t theta {int(degrees(theta))}°\t phi {int(degrees(phi))}°\t:\t{fidelity}')


In [20]:
print("Unmitigated Fidelities\n----------------------\n")

for count_idx, state_idx in zip(range(6, 33, 3), range(9)):
    print_fidelities(result_counts[count_idx], result_counts[count_idx + 1], result_counts[count_idx + 2], states[state_idx][0], states[state_idx][1])

Unmitigated Fidelities
----------------------

State fidelity for 	 theta -88°	 phi 79°	:	0.988098182078268
State fidelity for 	 theta 32°	 phi -167°	:	0.957588076628335
State fidelity for 	 theta 140°	 phi 125°	:	0.9758068206106877
State fidelity for 	 theta -76°	 phi -88°	:	0.9396590991235733
State fidelity for 	 theta 93°	 phi -135°	:	0.9984625580017882
State fidelity for 	 theta 104°	 phi -96°	:	0.9934786575712413
State fidelity for 	 theta -104°	 phi -146°	:	0.9460861290407586
State fidelity for 	 theta -158°	 phi -108°	:	0.973202251742156
State fidelity for 	 theta -110°	 phi -172°	:	0.9483355992495069


In [24]:
def get_mitigation_matrix(num_qubits: int, zero_counts: dict[str, int], one_counts: dict[str, int]) -> ndarray:

    assignment_matrices = []
    
    for qubit_idx in range(num_qubits):

        # Determine zero state for assignment matrix
        zero_count = sum([count for result, count in zero_counts.items() if result[qubit_idx] == '0'])
        one_count = sum([count for result, count in zero_counts.items() if result[qubit_idx] == '1'])

        # Calculate zero ket
        zero_ket = array([[zero_count], 
                          [one_count]]) / (zero_count + one_count)
        
        # Determine one state for assignment matrix
        zero_count = sum([count for result, count in one_counts.items() if result[qubit_idx] == '0'])
        one_count = sum([count for result, count in one_counts.items() if result[qubit_idx] == '1'])

        # Calculate zero ket
        one_ket = array([[zero_count], 
                         [one_count]]) / (zero_count + one_count)
        
        assignment_matrix = numpy.concatenate([zero_ket, one_ket], axis=1)

        assignment_matrices.append(assignment_matrix)

    return tens(*(scipy.linalg.inv(mat) for mat in assignment_matrices))


def get_bit_strings(count: int) -> list[str]:
    return [''.join(bits) for bits in itertools.product(['0', '1'], repeat=count)]


def mitigate_counts(num_qubits: int, mitigation_matrix: ndarray, counts: dict[str, int]) -> dict[str, int]:
    
    bit_strings = get_bit_strings(num_qubits)

    # Get vector of counts
    counts_vector = numpy.concatenate(
        [array([[counts[bit_string]] if bit_string in counts else 0.0]) for bit_string in bit_strings],
        axis=0
    )

    # Multiply by mitigation matirx
    corrected_vector = mitigation_matrix @ counts_vector
    corrected_vector = corrected_vector.astype(int)

    output = {}
    for idx, bit_string in zip(range(2**num_qubits), bit_strings):
        output[bit_string] = int(corrected_vector[idx])

    return output

In [ ]:
def print_mitigated_fidelities(x_counts, y_counts, z_counts, theta, phi):

    x_counts_mitigated = mitigate_counts(1, get_mitigation_matrix(1, qdt_x_counts[0], qdt_x_counts[1]), x_counts)
    y_counts_mitigated = mitigate_counts(1, get_mitigation_matrix(1, qdt_y_counts[0], qdt_y_counts[1]), y_counts)
    z_counts_mitigated = mitigate_counts(1, get_mitigation_matrix(1, qdt_z_counts[0], qdt_z_counts[1]), z_counts)

    den_op = density_op_from_counts_dict(x_counts_mitigated, y_counts_mitigated, z_counts_mitigated)

    fidelity = state_fidelity(den_op, gen_qubit(theta, phi))

    print(f'State fidelity for \t theta {int(degrees(theta))}°\t phi {int(degrees(phi))}°\t:\t{fidelity}')

In [26]:
def bloch_oordinate(counts_dict: dict) -> float:
    plus = counts_dict['0'] if '0' in counts_dict else 0
    minus = counts_dict['1'] if '1' in counts_dict else 0
    return (plus - minus) / (plus + minus)

In [28]:
print("Mitigated Fidelities\n----------------------\n")

for count_idx, state_idx in zip(range(6, 33, 3), range(9)):
    print_mitigated_fidelities(result_counts[count_idx], result_counts[count_idx + 1], result_counts[count_idx + 2], states[state_idx][0], states[state_idx][1])

Mitigated Fidelities
----------------------

State fidelity for 	 theta -88°	 phi 79°	:	0.9983992645128735
State fidelity for 	 theta 32°	 phi -167°	:	0.999983569494294
State fidelity for 	 theta 140°	 phi 125°	:	0.9987421432715281
State fidelity for 	 theta -76°	 phi -88°	:	0.9998955075961393
State fidelity for 	 theta 93°	 phi -135°	:	0.999925122773263
State fidelity for 	 theta 104°	 phi -96°	:	0.9995906827239042
State fidelity for 	 theta -104°	 phi -146°	:	0.999997188832673
State fidelity for 	 theta -158°	 phi -108°	:	0.999821278714556
State fidelity for 	 theta -110°	 phi -172°	:	0.9992339437947302
